# Labeling the Data
We identified many issues in Phase 1. Now we will go through and programatically label the records in this sample if they contain some of those issues.
## Author Problems
We'll start off by investigating the *author* field. This is an area that was found to have a number of potentially high priority issues as it pertains to social and political matters, as well as a field that has seen the some of the most pervasive issues in standardization. 

Start by importing the packages we'll need, setting up our directories, and loading in the data.

In [1]:
import pandas as pd #Creating dataframe and manipulating data
from bs4 import BeautifulSoup as bs # for cleaning xml tags
import re #regular expressions used for detection of initials
from py3langid.langid import LanguageIdentifier, MODEL_FILE #For language detection
from nltk.tokenize import sent_tokenize #Tokenizing abstracts during language detection
from pathlib import Path

In [2]:
# Data Directory
data_dir = Path('../data')
input_dir = data_dir / 'input'
output_dir = data_dir / 'output'
# Loading in dataset
df = pd.read_csv(input_dir / '02_cleaned_data.csv', 
                 usecols=['publisher', 'container-title', 'language', 'DOI', 'published', 
                          'created', 'deposited', 'title', 'author', 'abstract', 'original-title'],
                 parse_dates=['created', 'deposited'],
                 infer_datetime_format=True)
df.head()

/tmp/ipykernel_34727/2151301434.py:6: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(input_dir / '02_cleaned_data.csv',


,publisher,DOI,created,title,author,container-title,language,deposited,published,abstract,original-title
Index,,,,,,,,,,,
0,Wiley,10.1002/(sici)1099-1727(200021)16:1<27::aid-sd...,2002-09-10,The validation of commercial system dynamics m...,"[{'given': 'Geoff', 'family': 'Coyle', 'sequen...",System Dynamics Review,en,2021-07-01,2000.0,NaN,NaN
1,Springer Science and Business Media LLC,10.1007/bf02653972,2007-07-17,Effect of system geometry on the leaching beha...,"[{'given': 'C.', 'family': 'Vu', 'sequence': '...",Metallurgical Transactions B,en,2019-05-20,1979.0,NaN,NaN
2,Wiley,10.1111/reel.12221,2017-12-01,The international law on transboundary haze po...,"[{'given': 'Shawkat', 'family': 'Alam', 'seque...","Review of European, Comparative &amp; Internat...",en,2017-12-01,2017.0,NaN,NaN
3,Crop Science Society of Japan,10.1626/jcs.20.219,2011-09-20,Studies on the influence of pruning on the veg...,"[{'given': 'C.', 'family': 'TSUDA', 'sequence'...",Japanese Journal of Crop Science,en,2021-04-30,1951.0,NaN,NaN
4,Elsevier BV,10.1016/j.pneumo.2018.09.002,2018-10-10,Le tabagisme et l’aide à l’arrêt du tabac des ...,"[{'given': 'J.', 'family': 'Perriot', 'sequenc...",Revue de Pneumologie Clinique,fr,2019-10-26,2018.0,NaN,NaN


## Missing Values in Common Fields
This is a relatively easy problem to label, so we'll tackle these first.

We'll set up a column *'no_author'* and assign `0` to all of the records. Then we will locate the records missing an author and change their value to `1`.

Then we'll do the same for the *language, abstract,* and *title* fields.

In [3]:
#Authors
df['no_author'] = float(0)
df.loc[df.author.isna(), 'no_author'] = float(1)
#Language
df['no_language'] = float(0)
df.loc[df.language.isna(), 'no_language'] = float(1)
#Abstracts
df['no_abstract'] = float(0)
df.loc[df.abstract.isna(), 'no_abstract'] = float(1)
#Titles
df['no_title'] = float(0)
df.loc[df.title.isna(), 'no_title'] = float(1)

In [4]:
# Prevalence of missing values
#Missing Authors
prevalence_AuMis = (len(df.loc[df.no_author == 1])/len(df)) * 100
prevalence_AuMis # percentage of records with this specific issue

9.041

In [5]:
#Missing Language
prevalence_LangMis = (len(df.loc[df.no_language == 1])/len(df)) * 100
prevalence_LangMis

20.11

In [6]:
#Missing Abstract
prevalence_AbsMis = (len(df.loc[df.no_abstract == 1])/len(df)) * 100
prevalence_AbsMis

83.745

In [7]:
#Missing Title
prevalence_TitleMis = (len(df.loc[df.no_title == 1])/len(df)) * 100
prevalence_TitleMis

1.036

## Problem Detection Functions and Data Labeling
### Author Sequence
Our first function will be checking the *sequence* sub-field within the *author* field. This is the field wherein authors are either listed as 'first' or 'addtional'. This function sets up a counter then iterates through the author list of a record to check what the noted sequence is for each author.

The `try` block filters out records that have no authors listed. After that we begin to iterate through each author within a given record.

`If 'name' in author.keys():` is used to filter out institutions listed as authors as using the 'name' key is often how an institution is presented as an author within the metadata record. The code within the `if` block simply says if there's an institution as an author and they are the only author listed, increase the counter to 1, then the code will continue down to the `return` statements where **0** will be returned as technically there is not an issue with sequence in that record.

`else: if author['sequence'] == 'first'` block is where the bulk of the counting activity will happen. Up until this point we are mostly filtering out instances that don't apply to the problem at hand. Simply, the function will count how many authors are labled as 'first'. Once all authors of a record have been parsed, we go to the `return` statements.

In [8]:
def sequence_checker(authorList):
    counter = 0 
    try: 
        for author in authorList:
                if 'name' in author.keys():
                    if len(authorList) == 1:
                        counter +=1
                    else:
                        continue
                else:
                    if author['sequence'] == 'first':
                        counter +=1
                    else:
                        continue
        if counter == 0:
            return 1 #no first author
        elif len(authorList) > 1:
            if counter > 1:
                return 1 #multiple first authors
            else:
                return 0
        else:
            return 0 #no issue
    except:
        return None

In [9]:
# The 'author' and 'subject' columns need to be evaluated and formated before parsing,
# otherwise they are treated as strings instead of dicts/lists.
import ast
def reformat_col(record):
    try:
        formed = ast.literal_eval(record)
        return formed
    except:
        return None

cols_to_reformat = ['author']
for col in cols_to_reformat:
    df[col] = df[col].apply(lambda x: reformat_col(x))

In [10]:
df['author_sequence'] = df.author.map(lambda x: sequence_checker(x))

In [11]:
records_with_AuSeq = df.loc[(df.author_sequence == 1)] #creating a df with only the cords with these errors
prevalence_AuSeq = ((len(records_with_AuSeq))/(df.author.notnull().sum())) * 100
prevalence_AuSeq #returning a percent of the total number of records with this particular issue

1.1774535779856858

### Author Initials
This function will utilize regular expressions for detecting the use of initials. Specifically, we are looking for when initials are used in totality, that is to say a record with "Marianne E." will not be flagged, whereas a record with "D." will.

We look in both the 'given' and the 'family' sub-fields as this use of initials has been found in both sub-fields previously. 

The flow of the function operates similarly to the `sequence_checker`, we filter out records with `null` authors in the first `try` statement, followed by iteration through the author list, then another `try` statement where we filter out institutions as authors.

The regular expressions can be broken into two conditions: `^(?:[A-Z]\W{,3}\s?){,3}` and `(?:[^\W\d_.]\W){1,2}\B` which are seperated by `|`. This is because each of those expressions are looking for initials, the former is looking in ASCII characters, whereas the latter s looking for the pattern in non-Latin characters.

`if detector != None or len(author['given']) == 1` insures that all initialized names are caught and then returned with the appropriate label.

In [12]:
def author_initials_checker(authorList):
    try: #Filter for no authors
        for author in authorList: #iterating through author array
            try: #filter for institutions as authors
                detector = re.search(r"^(?:[A-Z]\W{,3}\s?){1,3}$", author['given']) #checking for initials in given
                if detector or len(author['given']) == 1:
                    return 1 #initials used
                else:
                    family_detector = re.search(r"^(?:[A-Z]\W{,3}\s?){1,3}$", author['family']) #initials in family
                    if family_detector or len(author['family']) == 1:
                        return 1 #initials used
                    else:
                        pass
            except:
                pass
                        
    except:
        return None
    return 0 #no issue

In [13]:
df['author_initials'] = df.author.map(lambda x: author_initials_checker(x))

In [14]:
records_with_initials = df.loc[df.author_initials == 1]
prevalence_author_initials = ((len(records_with_initials))/(df.author.notnull().sum())) * 100
prevalence_author_initials # percentage of records with this specific issue

26.50754735650128

### Institutions as Authors
This function will address instances in which institutions are recorded as authors.

`try:` will filter out records with `null` authors. Then we have the `institutions_present` list that looks for the telltale sign of an institution, the 'name' sub-field. 

If the list is populated with any authors, then the appropriate label signalling an institution will be returned.

In [15]:
def institution_as_author(authorList):
    try:
        institutions_present = [author for author in authorList if 'name' in author.keys()]
        if len(institutions_present) > 0:
            return 1 #institution as author
        else:
            return 0 #no issue
    except:
        return None

In [16]:
df['author_institutions'] = df.author.map(lambda x: institution_as_author(x))

In [17]:
records_with_AuIns = df.loc[df.author_institutions == 1]
prevalence_AuIns = ((len(records_with_AuIns))/(df.author.notnull().sum())) * 100
prevalence_AuIns #percentage of records with this specific issue

1.9393353049176003

### Affiliation Missing

This function will check if there is any data present within the Author `"Affiliation"` subfield.

We start by creating a variable to operate as a indicator to the presence of an affiliation. We then iterate through each author within a given record.

If an affiliation is present, we change the indicator to be `False`. After checking the authors, we assign `1` to records that are missing affiliations or `0` if there is no issue present.

In [18]:
def affiliations_missing(author_list):
    no_affil = True
    try:
        for author in author_list:
            affiliation = author['affiliation']
            if len(affiliation) == 0:
                continue
            else:
                no_affil = False
        if no_affil:
            return 1
        if not no_affil:
            return 0
    except:
        return None

In [19]:
df['affiliation_missing'] = df.author.map(lambda x: affiliations_missing(x))

In [20]:
records_missing_affil = df.loc[df.affiliation_missing == 1]
prevalence_miss_affil = ((len(records_missing_affil))/(df.author.notnull().sum())) * 100
prevalence_miss_affil

82.20846755131433

### Checking for Honorifics in Author Names

This function utilizes a set list of honorifics found, in Phase 1, to be used within the Author `"Given"` and `"Family"` subfields.

After establishing our list, we then iterate through each Author of every record, putting their names into a lowercase format.

We then check the given and family names for the use of the listed titles. Return `1` if an honorific is present. Return `0` if none are found.

In [21]:
def honorific_checker(author_list):
    titles_list = set(['dr.', 'prof', 'prof.', 'professor', 'doctor', 'dr', 'ingeniero'])
    try:
        for author in author_list:
            lowercase_given = author['given'].lower()
            lowercase_family = author['family'].lower()
            if any(word in titles_list for word in lowercase_given.split()):
                return 1
            elif any(word in titles_list for word in lowercase_family.split()):
                return 1
            else: 
                continue
        return 0
    except:
        return None

In [22]:
df['author_honorific'] = df.author.map(lambda x: honorific_checker(x))

In [23]:
records_with_honorific = df.loc[df.author_honorific == 1]
prevalence_honorific = ((len(records_with_honorific))/df.author.notnull().sum()) * 100
prevalence_honorific

0.09674688595960818

### Uppercase Author Names

With this function we will check each Author's `"Given"` and `"Family"` name subfields to see if the input is in all upercase letters.

We start iterating through the author list and filter out records wherein the number of characters in a `"Given"` name is `1`. These are likely to be initials and as such are covered by another dimension of issue detection. We then use the regular expression `(?:^[A-Z]+)$` to return matches when an Author's name is in all uppercase letters. 

If a match is found we return `1` to signifiy the existence of an issue. If no match is returned, we repeat the process using the author's `"Family"` name. If no match is found for the `"Family"` name, then we proceed with the next author within the record until all authors have been checked.

In [24]:
def uppercase_name(author_list):
    try:
        for author in author_list:
            if len(author['given']) == 1:
                continue
            else:
                if re.match(r'(?:^[A-Z]+)$', author['given']):
                    return 1
                else:
                    if len(author['family']) == 1:
                        continue
                    else:
                        if re.match(r'(?:^[A-Z]+)$', author['family']):
                            return 1
                        else:
                            continue
        return 0
    except:
        return None

In [25]:
df['author_uppercase'] = df.author.map(lambda x: uppercase_name(x))

In [26]:
records_uppercase = df.loc[df.author_uppercase == 1]
prevalence_uppercase = ((len(records_uppercase))/df.author.notnull().sum()) * 100
prevalence_uppercase

5.191349948878066

### Non-Latin Characters

This function detects the use of non-latin character sets. Particularly we are interested in practices of romanization and when it occurs: which journals, are the *language* fields present and accurate, and so on. 

First, we have to identify which records are using non-latin characters.

This is split into two different functions. The first utilizes a regular expression `(?:[^ı́\x00-\xff])` to detect any characters not in ISO-8859-1 (or Latin-1) (See note).

The second then utlizes the first function to then check each author within a given record.

Note: This expression is providing a few too many false positives for my liking. I'm currently working on a better expression or a different solution entirely.

In [27]:
def isLatinChar(text):
    regexp = re.compile(r'(?:[^ı́\x00-\xff])')
    if regexp.search(text):
        return True
    else:
        return False
def latin_script_checker(authorList):
    try:
        latin_scripts = [author for author in authorList if isLatinChar(author['given'])]
        if len(latin_scripts) > 0:
            return 1 # non-latin script found
        else:
            return 0 # no issue
    except:
        return None

In [28]:
df['author_characters'] = df.author.map(lambda x: latin_script_checker(x))

In [29]:
records_with_non_latin = df.loc[df.author_characters == 1]
prevalence_NonLatin = ((len(records_with_non_latin))/(df.author.notnull().sum())) * 100
prevalence_NonLatin #percentage of records with this specific issue

2.2251783770709883

### Abstract Multi-lingualism Detection
This function will detect the use of more than one language within the *abstract* field. As mentioned before, we're interested in how people pracitice recording metadata as it pertains to language.

Here we have a list of language ISO 639-1 codes. While it is not exhaustive (there are 183 offically assigned codes, and only 94 are present in this list), it does include many of the macrolanguages for which many other languages fall within.

We pass this list to `langid` to ensure a higher confidence intervals in it's identification, i.e. an abstract might be in Malay (ms) but the identifier might return 'ms' and 'id' (Indonesian) with lower confidence intervals for each. As Malay is the macrolanguage that covers Indonesian, we will keep 'ms' but not 'id'.

The first `try:` block filters for records without abstracts present, then we tokenize the abstracts by sentence.

Next we pick out the first sentence and the second to last sentence of each abstract. The reason for picking out the second to last sentence is because most occurences of multi-lingual abstracts are such that the abstract is first written in one language, and then a second time in another. The reason for not picking the last sentence is because it is not uncommon for footnotes or citations to be present at the end of the abstracts in these metadata records. The presence of these at the end of an abstract section make language detection problematic as the syntactical structure can be odd and leads to an incorrect detection.

We then classify both sentences, followed by an evaluation of the confidence intervals. If the confidence interval is especially low, it is omitted.

We then check to see if there is more than one language present in the dictionary with `len(set(lang_dict.keys()))`, if so the record is returned with a **1**, indicating and error. Otherwise it is returned with a **0**.

In [30]:
# Uncomment the two lines below if this is the first time running the script.
#import nltk
#nltk.download('punkt')
identifier = LanguageIdentifier.from_pickled_model(MODEL_FILE, norm_probs = True)
lang_list = ['af', 'am', 'ar', 'as', 'az', 'be', 'bg', 'bn', 'br', 
             'bs', 'ca', 'cs', 'cy', 'da', 'de', 'dz', 'el', 'en', 'eo', 
             'es', 'et', 'eu', 'fa', 'fi', 'fo', 'fr', 'ga', 'gl', 'gu', 
             'he', 'hi', 'hr', 'ht', 'hu', 'hy', 'is', 'it', 'ja', 'jv', 
             'ka', 'kk', 'km', 'kn', 'ko', 'ku', 'ky', 'la', 'lb', 'lo', 
             'lt', 'lv', 'mg', 'mk', 'ml', 'mn', 'mr', 'ms', 'mt', 'ne', 
             'nl', 'no', 'oc', 'or', 'pa', 'pl', 'ps', 'pt', 'qu', 'ro', 
             'ru', 'rw', 'se', 'si', 'sk', 'sl', 'sq', 'sr', 'sv', 'sw', 
             'ta', 'te', 'th', 'tl', 'tr', 'ug', 'uk', 'ur', 'vi', 'vo', 
             'wa', 'xh', 'zh', 'zu']
identifier.set_languages(langs=lang_list)
def lang_checker(abstract):
    try:
        # Tokenizing abstracts
        tokenized = sent_tokenize(abstract)
        startAndFinish = [tokenized[0], tokenized[-1]]
        # Detecting languages present
        lang = [identifier.classify(lang) for lang in startAndFinish]
        # Filter low confidence results
        lang_dict = {key:value for (key,value) in lang if value > .95}
        # Labeling specific issues found in record
        if len(set(lang_dict.keys())) > 1:
            return 1 #Multiple languages detected
    except:
        return None #No abstract
    return 0 #No issues

In [31]:
df['abstract_multi_lang']  = df.abstract.map(lambda x: lang_checker(x))

In [32]:
records_with_MultiLang = df.loc[(df.abstract_multi_lang == 1)]
prevalence_MultiLang = ((len(records_with_MultiLang))/(df.abstract.notnull().sum())) * 100
prevalence_MultiLang #returning a percent of the total number of records with this particular issue

1.7410027683789604

### Title Language Checking
This function will check the language of the title against the stated language of the record.

It is a relatively striaghtforward function: `try:` filters out records without a *title*, then classifies the language, and finally checks to see if the returned code matches what is record in the language field.

We use `df.apply` instead of `df.column.map` because of the need to check multiple fields within a record as opposed to being contained within a specific field.

Here, it should be mentioned, there is some abiguity. The *language* field is not clearly defined (is it the language of the Item, Container, or the record). The prevelance of this issue (seen below) reflects the lack of clarity in what this field is meant to represent.

In [33]:
def title_lang_checker(record):
    try:
        if str(record['title']).lower() != 'nan':
            lang = identifier.classify(record['title'])
            if lang[1] > .99:
                if lang[0] == record['language']:
                    return 0
                else:
                    if str(record['language']).lower() == 'nan':
                        return None
                    else:
                        return 1
            else:
                return None
        else:
            return None
    except:
        return None

In [34]:
df['title_language'] = df.apply(title_lang_checker, axis=1)

In [ ]:
records_with_TitleLang = df.loc[(df.title_language == 1)] #creating a df with only the records with these errors
prevalence_TitleLang = ((len(records_with_TitleLang))/len(df.loc[(df.title.notnull()) & (df.language.notnull())])) * 100
prevalence_TitleLang #returning a percent of the total number of records with this particular issue

3.1365731993046633

### Total Errors
Lastly, we'll add up all of the errors for each record and store them number in *'total_errors'* column.

In [ ]:
# Labled Columns
column_list = ['no_author', 'no_language', 'no_title', 'author_sequence', 'author_initials', 'author_institutions',
              'author_characters', 'abstract_multi_lang', 'title_language', 'author_uppercase',
              'affiliation_missing', 'author_honorific']
df['total_errors'] = df[column_list].sum(axis=1)

In [ ]:
#Taking a look at the df
df.head()

,publisher,DOI,created,title,author,container-title,language,deposited,published,abstract,...,author_sequence,author_initials,author_institutions,affiliation_missing,author_honorific,author_uppercase,author_characters,abstract_multi_lang,title_language,total_errors
Index,,,,,,,,,,,,,,,,,,,,,
0,Wiley,10.1002/(sici)1099-1727(200021)16:1<27::aid-sd...,2002-09-10,The validation of commercial system dynamics m...,"[{'given': 'Geoff', 'family': 'Coyle', 'sequen...",System Dynamics Review,en,2021-07-01,2000.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,1.0
1,Springer Science and Business Media LLC,10.1007/bf02653972,2007-07-17,Effect of system geometry on the leaching beha...,"[{'given': 'C.', 'family': 'Vu', 'sequence': '...",Metallurgical Transactions B,en,2019-05-20,1979.0,NaN,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,2.0
2,Wiley,10.1111/reel.12221,2017-12-01,The international law on transboundary haze po...,"[{'given': 'Shawkat', 'family': 'Alam', 'seque...","Review of European, Comparative &amp; Internat...",en,2017-12-01,2017.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,1.0
3,Crop Science Society of Japan,10.1626/jcs.20.219,2011-09-20,Studies on the influence of pruning on the veg...,"[{'given': 'C.', 'family': 'TSUDA', 'sequence'...",Japanese Journal of Crop Science,en,2021-04-30,1951.0,NaN,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,3.0
4,Elsevier BV,10.1016/j.pneumo.2018.09.002,2018-10-10,Le tabagisme et l’aide à l’arrêt du tabac des ...,"[{'given': 'J.', 'family': 'Perriot', 'sequenc...",Revue de Pneumologie Clinique,fr,2019-10-26,2018.0,NaN,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,2.0


In [ ]:
df.to_csv(output_dir / '03_labeled_data.csv', index=False)